In [2]:
import random
import pickle
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [3]:
text_df = pd.read_fwf("input.txt")

In [4]:
text_df

,First Citizen
0,Before we proceed any further hear me speak
1,All
2,Speak speak
3,First Citizen
4,You are all resolved rather to die than to famish
...,...
32771,And yet so fast asleep
32772,ANTONIO
32773,Noble Sebastian
32774,Thou let'st thy fortune sleep--die rather wink'st


In [38]:
with open("input.txt", "r") as file:
    text = file.readlines()
print(text)


['First Citizen\n', 'Before we proceed any further hear me speak\n', '\n', 'All\n', 'Speak speak\n', '\n', 'First Citizen\n', 'You are all resolved rather to die than to famish\n', '\n', 'All\n', 'Resolved resolved\n', '\n', 'First Citizen\n', 'First you know Caius Marcius is chief enemy to the people\n', '\n', 'All\n', "We know't we know't\n", '\n', 'First Citizen\n', "Let us kill him and we'll have corn at our own price\n", "Is't a verdict\n", '\n', 'All\n', "No more talking on't let it be done away away\n", '\n', 'Second Citizen\n', 'One word good citizens\n', '\n', 'First Citizen\n', 'We are accounted poor citizens the patricians good\n', 'What authority surfeits on would relieve us if they\n', 'would yield us but the superfluity while it were\n', 'wholesome we might guess they relieved us humanely\n', 'but they think we are too dear the leanness that\n', 'afflicts us the object of our misery is as an\n', 'inventory to particularise their abundance our\n', 'sufferance is a gain to 

text

In [39]:
text = [line.strip() for line in text if line.strip() and len(line.split()) > 1]

In [40]:
text

['First Citizen',
 'Before we proceed any further hear me speak',
 'Speak speak',
 'First Citizen',
 'You are all resolved rather to die than to famish',
 'Resolved resolved',
 'First Citizen',
 'First you know Caius Marcius is chief enemy to the people',
 "We know't we know't",
 'First Citizen',
 "Let us kill him and we'll have corn at our own price",
 "Is't a verdict",
 "No more talking on't let it be done away away",
 'Second Citizen',
 'One word good citizens',
 'First Citizen',
 'We are accounted poor citizens the patricians good',
 'What authority surfeits on would relieve us if they',
 'would yield us but the superfluity while it were',
 'wholesome we might guess they relieved us humanely',
 'but they think we are too dear the leanness that',
 'afflicts us the object of our misery is as an',
 'inventory to particularise their abundance our',
 'sufferance is a gain to them Let us revenge this with',
 'our pikes ere we become rakes for the gods know I',
 'speak this in hunger for 

In [41]:
text = list (text)

In [42]:
text

['First Citizen',
 'Before we proceed any further hear me speak',
 'Speak speak',
 'First Citizen',
 'You are all resolved rather to die than to famish',
 'Resolved resolved',
 'First Citizen',
 'First you know Caius Marcius is chief enemy to the people',
 "We know't we know't",
 'First Citizen',
 "Let us kill him and we'll have corn at our own price",
 "Is't a verdict",
 "No more talking on't let it be done away away",
 'Second Citizen',
 'One word good citizens',
 'First Citizen',
 'We are accounted poor citizens the patricians good',
 'What authority surfeits on would relieve us if they',
 'would yield us but the superfluity while it were',
 'wholesome we might guess they relieved us humanely',
 'but they think we are too dear the leanness that',
 'afflicts us the object of our misery is as an',
 'inventory to particularise their abundance our',
 'sufferance is a gain to them Let us revenge this with',
 'our pikes ere we become rakes for the gods know I',
 'speak this in hunger for 

In [43]:
joined_text = " ".join(text)

In [44]:
joined_text

"First Citizen Before we proceed any further hear me speak Speak speak First Citizen You are all resolved rather to die than to famish Resolved resolved First Citizen First you know Caius Marcius is chief enemy to the people We know't we know't First Citizen Let us kill him and we'll have corn at our own price Is't a verdict No more talking on't let it be done away away Second Citizen One word good citizens First Citizen We are accounted poor citizens the patricians good What authority surfeits on would relieve us if they would yield us but the superfluity while it were wholesome we might guess they relieved us humanely but they think we are too dear the leanness that afflicts us the object of our misery is as an inventory to particularise their abundance our sufferance is a gain to them Let us revenge this with our pikes ere we become rakes for the gods know I speak this in hunger for bread not in thirst for revenge Second Citizen Would you proceed especially against Caius Marcius Aga

In [45]:
#partial_split = joined_text[:10000]
partial_split = joined_text

In [46]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_split.lower())

In [47]:
tokens

['first',
 'citizen',
 'before',
 'we',
 'proceed',
 'any',
 'further',
 'hear',
 'me',
 'speak',
 'speak',
 'speak',
 'first',
 'citizen',
 'you',
 'are',
 'all',
 'resolved',
 'rather',
 'to',
 'die',
 'than',
 'to',
 'famish',
 'resolved',
 'resolved',
 'first',
 'citizen',
 'first',
 'you',
 'know',
 'caius',
 'marcius',
 'is',
 'chief',
 'enemy',
 'to',
 'the',
 'people',
 'we',
 'know',
 't',
 'we',
 'know',
 't',
 'first',
 'citizen',
 'let',
 'us',
 'kill',
 'him',
 'and',
 'we',
 'll',
 'have',
 'corn',
 'at',
 'our',
 'own',
 'price',
 'is',
 't',
 'a',
 'verdict',
 'no',
 'more',
 'talking',
 'on',
 't',
 'let',
 'it',
 'be',
 'done',
 'away',
 'away',
 'second',
 'citizen',
 'one',
 'word',
 'good',
 'citizens',
 'first',
 'citizen',
 'we',
 'are',
 'accounted',
 'poor',
 'citizens',
 'the',
 'patricians',
 'good',
 'what',
 'authority',
 'surfeits',
 'on',
 'would',
 'relieve',
 'us',
 'if',
 'they',
 'would',
 'yield',
 'us',
 'but',
 'the',
 'superfluity',
 'while',
 'it

In [48]:
unique_tokens = np.unique(tokens)

In [49]:
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [50]:
unique_token_index

{'3': 0,
 'a': 1,
 'abandon': 2,
 'abase': 3,
 'abate': 4,
 'abated': 5,
 'abbey': 6,
 'abbot': 7,
 'abed': 8,
 'abel': 9,
 'abet': 10,
 'abhor': 11,
 'abhorr': 12,
 'abhorred': 13,
 'abhorring': 14,
 'abhors': 15,
 'abhorson': 16,
 'abide': 17,
 'abides': 18,
 'abilities': 19,
 'ability': 20,
 'abject': 21,
 'abjects': 22,
 'abjured': 23,
 'able': 24,
 'aboard': 25,
 'abode': 26,
 'abodements': 27,
 'aboding': 28,
 'abominable': 29,
 'abortive': 30,
 'abound': 31,
 'about': 32,
 'above': 33,
 'abraham': 34,
 'abreast': 35,
 'abroach': 36,
 'abroad': 37,
 'absence': 38,
 'absent': 39,
 'absolute': 40,
 'absolutely': 41,
 'absolved': 42,
 'absolver': 43,
 'abstains': 44,
 'abstinence': 45,
 'abstract': 46,
 'abundance': 47,
 'abundant': 48,
 'abundantly': 49,
 'abuse': 50,
 'abused': 51,
 'abuses': 52,
 'abusing': 53,
 'abysm': 54,
 'accent': 55,
 'accents': 56,
 'accept': 57,
 'acceptance': 58,
 'access': 59,
 'accessary': 60,
 'accident': 61,
 'accidental': 62,
 'accidentally': 63,
 '

In [51]:
number_words = 10
input_words = []
next_words = []

In [52]:
for i in range(len(tokens)-number_words):
  input_words.append(tokens[i:i + number_words])
  next_words.append(tokens[i + number_words])

In [53]:
input_words

[['first',
  'citizen',
  'before',
  'we',
  'proceed',
  'any',
  'further',
  'hear',
  'me',
  'speak'],
 ['citizen',
  'before',
  'we',
  'proceed',
  'any',
  'further',
  'hear',
  'me',
  'speak',
  'speak'],
 ['before',
  'we',
  'proceed',
  'any',
  'further',
  'hear',
  'me',
  'speak',
  'speak',
  'speak'],
 ['we',
  'proceed',
  'any',
  'further',
  'hear',
  'me',
  'speak',
  'speak',
  'speak',
  'first'],
 ['proceed',
  'any',
  'further',
  'hear',
  'me',
  'speak',
  'speak',
  'speak',
  'first',
  'citizen'],
 ['any',
  'further',
  'hear',
  'me',
  'speak',
  'speak',
  'speak',
  'first',
  'citizen',
  'you'],
 ['further',
  'hear',
  'me',
  'speak',
  'speak',
  'speak',
  'first',
  'citizen',
  'you',
  'are'],
 ['hear',
  'me',
  'speak',
  'speak',
  'speak',
  'first',
  'citizen',
  'you',
  'are',
  'all'],
 ['me',
  'speak',
  'speak',
  'speak',
  'first',
  'citizen',
  'you',
  'are',
  'all',
  'resolved'],
 ['speak',
  'speak',
  'speak',
 

In [54]:
next_words

['speak',
 'speak',
 'first',
 'citizen',
 'you',
 'are',
 'all',
 'resolved',
 'rather',
 'to',
 'die',
 'than',
 'to',
 'famish',
 'resolved',
 'resolved',
 'first',
 'citizen',
 'first',
 'you',
 'know',
 'caius',
 'marcius',
 'is',
 'chief',
 'enemy',
 'to',
 'the',
 'people',
 'we',
 'know',
 't',
 'we',
 'know',
 't',
 'first',
 'citizen',
 'let',
 'us',
 'kill',
 'him',
 'and',
 'we',
 'll',
 'have',
 'corn',
 'at',
 'our',
 'own',
 'price',
 'is',
 't',
 'a',
 'verdict',
 'no',
 'more',
 'talking',
 'on',
 't',
 'let',
 'it',
 'be',
 'done',
 'away',
 'away',
 'second',
 'citizen',
 'one',
 'word',
 'good',
 'citizens',
 'first',
 'citizen',
 'we',
 'are',
 'accounted',
 'poor',
 'citizens',
 'the',
 'patricians',
 'good',
 'what',
 'authority',
 'surfeits',
 'on',
 'would',
 'relieve',
 'us',
 'if',
 'they',
 'would',
 'yield',
 'us',
 'but',
 'the',
 'superfluity',
 'while',
 'it',
 'were',
 'wholesome',
 'we',
 'might',
 'guess',
 'they',
 'relieved',
 'us',
 'humanely',
 'b

In [55]:
X = np.zeros((len(input_words),number_words,len(unique_tokens)), dtype = bool)

In [56]:
Y = np.zeros((len(next_words),len(unique_tokens)), dtype = bool)

In [57]:
X

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [58]:
Y

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [59]:
for i,words in enumerate(input_words):
  for j,word in enumerate(words):
    X[i,j,unique_token_index[word]] = 1
  Y[i,unique_token_index[next_words[i]]]=1

In [60]:
X,Y

(array([[[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],
 
        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],
 
        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, 

In [2]:
model = Sequential()
model.add(LSTM(128,input_shape=(number_words,len(unique_tokens)),return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

NameError: name 'Sequential' is not defined

In [1]:
model.compile(loss="categorical_crossentropy",optimizer=RMSprop(learning_rate=0.01),metrics=["accuracy"])
model.fit(X,Y,batch_size=128,epochs=30,shuffle=True)

NameError: name 'model' is not defined

In [ ]:
model.save("second_all_data.h5")

In [32]:
model = load_model("second_all_data.h5")

OSError: No file or directory found at second_all_data.h5

In [33]:
def predict_word(input_text,best):
  input_text = input_text.lower()
  X = np.zeros((1,number_words,len(unique_tokens)))
  for i,word in enumerate(input_text.split()):
    X[0,i,unique_token_index[word]]=1

  predictions = model.predict(X)[0]
  return np.argpartition(predictions, -best)[-best:]

In [ ]:
sen = predict_word("You are all resolved rather to die than to",5)

1/1 [==============================] - 0s 21ms/step


In [ ]:
print([unique_tokens[inx] for inx in sen])

['too', 'famish', 'you', 'resolved', 'marcius']


In [ ]:
def txt_gen(input_text,txt_len,creativity=3):
  word_sequence = input_text.split()
  current = 0
  for _ in range(txt_len):
    sub_sequence = "" .join(tokenizer.tokenize("".join(word_sequence).lower())[current:current+number_words])
    try:
      choice = unique_tokens[random.choice(predict_word(sub_sequence,creativity))]
    except:
      choice = random.choice(unique_tokens)
    word_sequence.append(choice)
    current+=1
  return " ".join(word_sequence)



In [ ]:
txt_gen("You are all resolved rather to die than to",100,10)

1/1 [==============================] - 0s 16ms/step


'You are all resolved rather to die than to course tell friends no says almost must these who must friends friends friends says who i these these you almost almost no these almost almost who i says friends you you no says you you no you who no who i says no these friends says says tell says must no must no who you friends who tell friends you almost must friends friends who no says i must who tell no tell tell these you says tell these no no friends tell i i says these you these who no no almost almost must must friends i who you'